## Filtering

Materials are based on [Neural Signal Processing (UCSD COGS118C)](https://github.com/rdgao/COGS118C) by Richard Gao (see the [LICENSE](https://github.com/rdgao/COGS118C/blob/master/LICENSE)).

In [1]:
# make the imports
import numpy as np
from scipy import io, signal
import bokeh, bokeh.io, bokeh.plotting # for fancy plots
bokeh.io.output_notebook()

### Load some real brain signals
We'll be working with the LFP data recorded in the rat hippocampus. This dataset comes from an openly accessible neuroscience database. For more information on this particular dataset, see [here](https://crcns.org/data-sets/hc/hc-2/about-hc-2).

In [ ]:
# Download the dataset (to Virtual Machine local filesystem)
# note the leading !. This is a signal to execute the line as a shell command
!wget https://krzysztof.kutt.pl/didactics/psaw/LFP.mat

In [3]:
data = io.loadmat('LFP.mat', squeeze_me=True)
print(data.keys())

dict_keys(['__header__', '__version__', '__globals__', 'lfp', 'fs', 'spike_indices', 'spike_fs'])


In [ ]:
# Przypomnienie:
# dodajemy wartosci kolumny df['fs'] - zmienne do fs
# Przypomnienie Sampling Rate - liczba sampli na sekunde wzietych z sygnalu ciaglego
# aby stworzyc sygnal cyfrowy (czyli dyskretny)
#


fs = data['fs'] # sampling rate 
print(f'Sampling rate = {fs} Hz')

lfp = data['lfp'][0,:]/1000 # this file contains two channels, we'll only work with the first one
lfp_short = lfp[:int(120*fs)] # make a variable that has the first two minutes of the LFP
t_short = np.arange(0, len(lfp_short)/fs, 1/fs) # create the corresponding time vector

# Przypomnienie: ( analogicznie do notebooka EEG ...)
# stworz odpowiedni wektor czasu
# tworzy tablice numpy od zera do len( lfp_short ) / fs, postepujaca co 1/fs ...

# Pomocnicze:
# The local field potential (LFP) refers to the electric potential in the extracellular space around neurons. 
# The LFP is a widely available signal in many recording configurations, 
# ranging from single-electrode recordings to multi-electrode arrays. 

bf = bokeh.plotting.figure(
    x_axis_label='Time (s)',
    y_axis_label='Voltage',
    plot_width=900,
    plot_height=250)
bf.line(t_short, lfp_short)
bf.x_range = bokeh.models.Range1d(0, 10)

bokeh.plotting.show(bf)

### Filtering
There are 4 types of filters: **lowpass**, **highpass**, **bandpass**, and **bandstop**. They refer to the frequency response of the filter, e.g., lowpass means to allow low frequencies through (from 0Hz to the cutoff) and filter out high frequencies, the opposite for highpass. Bandpass allows through a narrow band frequency and filters out the rest, while bandstop does the opposite, which is commonly used for filtering out a specific frequency of noise (such as 50 Hz line noise in Poland). Filter design is an art that will take many such labs to cover extensively, so we will just introduce the idea here and give you the tools to explore that at a later stage.

#### Finite Impulse Response 
The filter response can be examined in both time and frequency domain. If we plot the coefficients of an FIR filter in time, that's quite literally its impulse response function (IRF), i.e., if you tried to filter a delta with this function, it will output itself. FIR stands for finite impulse response, which means the impulse response function has finite time. Infinite impulse response (IIR) filters, on the other hand, have feedback, and thus will continue out to infinity even for a single delta input. Here, we will only use FIR filters.

#### `signal.firwin()`
A quick tutorial on `signal.firwin()`: this function designs the FIR filter based on your frequency requirements, and return the filter coefficients. The 4 critical parameters are `numtaps`, `fs`, `cutoff`, and `pass_zero`. 

- `numtaps` is the filter "order", basically, how many points are in the filter. The longer the filter is, the better frequency resolution you will have, but worse temporal resolution.
- `fs` is the sampling rate of your signal
- `cutoff` defines the frequency to pass/block
- `pass_zero` defines whether 0Hz is passed or blocked; it can also be a string argument for the desired filter type: `'bandpass', 'lowpass', 'highpass', 'bandstop'`

`cutoff` and `pass_zero`, in conjunction, defines the filter type. If your cutoff is at 20Hz and define `pass_zero=True`, then `firwin()` interprets that to be a lowpass filter. If `pass_zero=False`, then it's a highpass filter. Same idea applies to bandpass and bandstop, except `cutoff` is now required to be a tuple.

#### `np.convolve()`
Finally, to apply the filter, we simply convolve the signal with the filter, using `np.convolve()`. Remember, convolution in time domain is multiplication in frequency domain.

Below, there is a demonstration of an example lowpass filter, with its IRF plotted in time and frequency.

In [ ]:
#! Komentarze dla siebie 


# tupla z dwoma wybranymi wartosciami (indeksujemy jak listy pythonowe)

cutoff = (20, 40) #Hz

# Wytlumaczenie dla siebie: 
# Chcemy aby rzad filtra byl co najmniej tak dlugi jak 3 okresy
# najmniejszej czestotliwosci w tupli cutoff
# no to skoro cutoff ma powiedzmy 20Hz (czyli 20 cykli na sekunde) chcemy 3 * 0.05 sek = 0.15 sek dlugosci
# a to jest 0.15 * fs punktow
# im wieksza bedzie ta liczba tym wieksza rozdzielczosc czestotliwosci otrzymasz

# Na podstawie tego podstawiamy:
# (nasz filter dolnopasmowy)
filt_order = int(3*fs/cutoff[0])
filt_coefs = signal.firwin(filt_order, cutoff[0], fs=fs, pass_zero='lowpass')

# Magnitude Response - charakterystyka opisujaca jakie pasma czestotliwosci
# Filter blokuje albo przepuszcza

freq_resp = np.fft.fft(filt_coefs, n=int(fs)) # jednowymiarowa dyskretna transformata fouriera
mag_resp = abs(freq_resp)**2                  # wyliczamy mag response podnoszac do kwadratu odpowiedz czestotliwosciowa
ph_resp = np.angle(freq_resp)
freqs = np.fft.fftfreq(int(fs),1/fs)          # zwraca "pudelka" czestotliwosci dla zadanych parametrow fft

# mag_resp = abs(freq_resp) ** 2 - wzor analogiczny do power spectrum
# Przypomnienie z poprzedniego noteboooka:
# ... the power spectrum of a signal, which is the squared magnitude of the complex vector at every frequency.  ...

##### Plot 1: Impulse Response #####

# odpowiedz Impulsowa Filtra

# tworzymy wektor czasu dla wykresu
t_filt = np.arange(0,len(filt_coefs))/fs

bf1 = bokeh.plotting.figure(
    title='Impulse Response',
    x_axis_label='Time (s)',
    y_axis_label='Amplitude',
    plot_width=300,
    plot_height=250)
bf1.line(t_filt, filt_coefs)

##### Plot 2: Magnitude Response #####

bf2 = bokeh.plotting.figure(
    title='Magnitude Response',
    x_axis_label='Frequency (Hz)',
    y_axis_label='Magnitude',
    plot_width=300,
    plot_height=250)
bf2.line(freqs, mag_resp)
bf2.x_range = bokeh.models.Range1d(0, 100)

##### Plot 3: Phase Response #####

bf3 = bokeh.plotting.figure(
    title='Phase Response',
    x_axis_label='Frequency (Hz)',
    y_axis_label='Phase (rad)',
    plot_width=300,
    plot_height=250)
bf3.line(freqs, ph_resp)
bf3.x_range = bokeh.models.Range1d(0, 100)

##### Plot 4: Original and Filtered signals #####

lfp_filt = np.convolve(lfp_short, filt_coefs, mode='same')

bf4 = bokeh.plotting.figure(
    title='Low-Pass 20 Hz',
    x_axis_label='Time (s)',
    y_axis_label='Voltage (V)',
    plot_width=900,
    plot_height=250)
bf4.line(t_short, lfp_short,
         legend_label='Original', color='blue')
bf4.line(t_short, lfp_filt,
         legend_label='Filtered', color='red')
bf4.x_range = bokeh.models.Range1d(0, 1)


# show all plots

first_line = bokeh.layouts.row(bf1, bf2, bf3)
both_lines = bokeh.layouts.column(first_line, bf4)
bokeh.plotting.show(both_lines)


# 
# 



### Experiments with filters

**Task 1:** Following the template from above, do the following:
- construct filter coefficients using `signal.firwin()`
- plot the IRF in time and frequency domain (both magnitude and phase)
- filter the LFP signal using `np.convolve`, and plot both signals in time (zoom into first 5 seconds)
- plot the power spectrum of both the original and filtered signal.

There are three filters to be constructed:
1. a band-pass filter, with cut-off between 4-12Hz.
1. a band-stop filter, with cut-off between 4-12Hz.
1. a high-pass filter, with a cut-off at 1Hz.

Finally, answer the question:
4. Which of the above is most suitable for isolating (keeping) the dominant frequency in the LFP.

In [ ]:
# Filtr Bandpass
# Filtr Srodkowo Przepustowy  -
# Filtr przepuszczający składowe widmowe sygnału w określonym przedziale częstotliwości, nazywanym pasmem przepustowym. 

# wiec tworzymy tuple dla 4 i 12
cutoff1 = (4, 12) #Hz

# no to skoro cutoff ma powiedzmy 4Hz (czyli 4 cykli na sekunde) chcemy 3 * 0.25 sek = 0.75 sek dlugosci


filt_order = int(3*fs/10)
filt_coefs = signal.firwin(filt_order, cutoff1, fs=fs, pass_zero='bandpass')

# polcz magnitude oraz phase response dla filtra

freq_resp = np.fft.fft(filt_coefs, n=int(fs))
mag_resp = abs(freq_resp)**2
ph_resp = np.angle(freq_resp)
freqs = np.fft.fftfreq(int(fs),1/fs)


# Wykres Napiecia w czasie dla sygnalu przed i po uzyciu filtra
lfp_filt = np.convolve(lfp_short, filt_coefs, mode='same')

bf4 = bokeh.plotting.figure(
    title='Band-Pass 4 Hz',
    x_axis_label='Time (s)',
    y_axis_label='Voltage (V)',
    plot_width=900,
    plot_height=250)
bf4.line(t_short, lfp_short,
         legend_label='Sygnal Orginalny', color='blue')
bf4.line(t_short, lfp_filt,
         legend_label='Sygnal Po Uzyciu Filtra', color='red')
bf4.x_range = bokeh.models.Range1d(0, 1)

bokeh.plotting.show(bf4)

# Wykres power spektrum przed bandpass


# Wykres power spektrum po uzyciu bandpass

np_power = np.abs(mag_resp)**2

bf = bokeh.plotting.figure(
    title='Band-Pass Power Spectrum 4 Hz',
    x_axis_label='Frequency (Hz)',
    y_axis_label='Power (uV^2)',
    plot_width=400,
    plot_height=400)

bf.line(freqs, np_power)
bf.x_range = bokeh.models.Range1d(0, 50)

bokeh.plotting.show(bf)





# Filter bandstop
# Definicja:
# Filtr Bandstop - Filtr srodkowozaporowy -  
# filtr nieprzepuszczający częstotliwości sygnału między dwiema ustalonymi wartościami granicznymi.

cutoff2 = (4,12) #Hz

filt_order2 = int(3*fs/cutoff2[0])
filt_coefs2 = signal.firwin(filt_order2, cutoff2, fs=fs, pass_zero='bandstop')

# magnitude response i phase response dla filtra bandstop
freq_resp2 = np.fft.fft(filt_coefs2, n=int(fs))
mag_resp2 = abs(freq_resp2)**2
ph_resp2 = np.angle(freq_resp2)
freqs2 = np.fft.fftfreq(int(fs),1/fs)

# Wykres Napiecia w czasie dla sygnalu przed i po uzyciu filtra
lfp_filt2 = np.convolve(lfp_short, filt_coefs2, mode='same')

bf5 = bokeh.plotting.figure(
    title='Band-Stop 4 Hz',
    x_axis_label='Time (s)',
    y_axis_label='Voltage (V)',
    plot_width=900,
    plot_height=250)
bf5.line(t_short, lfp_short,
         legend_label='Sygnal Orginalny', color='blue')
bf5.line(t_short, lfp_filt2,
         legend_label='Sygnal Po Uzyciu Filtra', color='red')
bf5.x_range = bokeh.models.Range1d(0, 1)

bokeh.plotting.show(bf5)



# Wykres power spektrum po uzyciu bandstop

np_power2 = np.abs(mag_resp2)**2

bf6 = bokeh.plotting.figure(
    title='Band-Stop Power Spectrum 4 Hz',
    x_axis_label='Frequency (Hz)',
    y_axis_label='Power (uV^2)',
    plot_width=400,
    plot_height=400)

bf6.line(freqs2, np_power2)
bf6.x_range = bokeh.models.Range1d(0, 50)

bokeh.plotting.show(bf6)


# Filter HighPass
# Definicja:
# Filtr Gorno Przepustowy
# Filtr ktory przepuszcza sygnały sinusoidalne
# o częstotliwościach powyżej określonej częstotliwości granicznej, a tłumi składowe leżące poniżej tej częstotliwości.

cutoff3=1


filt_order3 = int(3*fs/10)
filt_coefs3 = signal.firwin(filt_order, 1, fs=fs, pass_zero='highpass')


freq_resp3 = np.fft.fft(filt_coefs3, n=int(fs))
mag_resp3 = abs(freq_resp3)**2
ph_resp3 = np.angle(freq_resp3)
freqs3 = np.fft.fftfreq(int(fs),1/fs)


# Wykres Napiecia w czasie dla sygnalu przed i po uzyciu filtra gornoprzepustowego


lfp_filt3 = np.convolve(lfp_short, filt_coefs3, mode='same')

bf7 = bokeh.plotting.figure(
    title='High Pass 1 Hz',
    x_axis_label='Time (s)',
    y_axis_label='Voltage (V)',
    plot_width=900,
    plot_height=250)
bf7.line(t_short, lfp_short,
         legend_label='Sygnal Orginalny', color='blue')
bf7.line(t_short, lfp_filt3,
         legend_label='Sygnal Po Uzyciu Filtra', color='red')
bf7.x_range = bokeh.models.Range1d(0, 1)

bokeh.plotting.show(bf7)





# Wykres power spektrum po uzyciu Filtra gornoprzepustowego

np_power3 = np.abs(mag_resp3)**2

bf8 = bokeh.plotting.figure(
    title='High Pass Power Spectrum 1 Hz',
    x_axis_label='Frequency (Hz)',
    y_axis_label='Power (uV^2)',
    plot_width=400,
    plot_height=400)

bf8.line(freqs2, np_power3)
bf8.x_range = bokeh.models.Range1d(0, 50)

bokeh.plotting.show(bf8)


# Podsumowanie:
# Pominalem Wszystkie wykresy dla odpowiedzi sygnalu
#  


**Response (for Task 1, point 4):** `#_FILL_IN_YOUR_RESPONSE_HERE`


In [ ]:
# Dominant Frequency - highest magnitude sinusoidal component of electrogram

# Obserwujac Tylko Wykresy Dla Uzycia powyzszych Filtrow intuicja podowiada 
# ze Filtr Gornoprzepustowy wyizoluje Dominujace Czestotliwosci dla tego sygnalu.


# Rozwiniecie na podstawie lektury paru artykulow (zrodla podane..)

# Badany sygnal to LFP - Local Field Potential
# Pomimo ze intuicja podpowiada Uzycie filtra gornoprzepustowego to
# po dokladniejszej lekturze
# ...
# http://www.scholarpedia.org/article/Local_field_potential\

# Mozna Dostrzec potencjal zastosowania takze filtra dolnoprzepustowego do uzyskiwania okreslonych
# czestotliwosci

# Zaglebiajac sie dalej:
# https://www.sciencedirect.com/science/article/pii/S0896627319301746

# Na podstawie lektury tego artykulu - potencjal maja takze filtry band pass
# 


